In [4]:
import pandas as pd
import os

def concat_csvs_by_date(file_list):
    # Read and store dataframes
    dfs = [pd.read_csv(file) for file in file_list]
    
    # Concat all dataframes
    combined_df = pd.concat(dfs, axis=1)
    
    # Remove duplicate Date columns, keep first
    combined_df = combined_df.loc[:,~combined_df.columns.duplicated()]
    
    # Save combined dataframe
    combined_df.to_csv('input.d.csv', index=False)
    print("Combined data saved to input.d.csv")

def create_time_lagged_features(input_file='input.d.csv'):
    # Read the input file
    df = pd.read_csv(input_file)
    
    # Convert Date to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Sort by Date
    df = df.sort_values('Date')
    
    # Create time-lagged versions
    time_lags = [10, 2, 1, 0]
    
    # Store dataframes
    lagged_dfs = []
    
    for lag in time_lags:
        # Create a copy of the dataframe
        lagged_df = df.copy()
        
        # Shift the columns (except Date)
        date_cols = df.columns[df.columns != 'Date']
        lagged_df[date_cols] = lagged_df[date_cols].shift(lag)
        
        # Rename columns to indicate lag
        lagged_cols = {col: f'{col}_lag_{lag}' for col in date_cols}
        lagged_df = lagged_df.rename(columns=lagged_cols)
        
        lagged_dfs.append(lagged_df)
    
    # Concatenate all time-lagged dataframes
    final_df = pd.concat(lagged_dfs, axis=1)
    
    # Remove duplicate Date columns, keep first
    final_df = final_df.loc[:,~final_df.columns.duplicated()]
    
    # Shift 'EWJ.Percent.Price' forward to t+1
    ewj_t_plus_1 = final_df['EWJ.Percent.Price_lag_1']
    final_df['EWJ.Percent.Price_forward_1'] = ewj_t_plus_1
    
    # Save the final dataframe
    final_df.to_csv('input.engineered.csv', index=False)
    print("Engineered features saved to input.engineered.csv")

files = [
    'etf_momentum_delta.csv',
    'NYSE.market.vol.csv',
    'US.JPN.macro.indicators.csv',
    'Toyota_price_change.csv',
    'Mitsubishi_UFJ_Financial_Group_price_change.csv',
    'Sony_price_change.csv',
    'Hitachi_price_change.csv',
    'R_Holdings_price_change.csv',
    'Sumitomo_Mitsui_Financial_Group_price_change.csv',
    'Keyence_price_change.csv',
    'Tokio_Marine_Holdings_price_change.csv',
    'Tokyo_Electron_price_change.csv',
    'Shin_Etsu_Chemical_price_change.csv',
    'Nikkei_225_price_metrics.csv'
]

concat_csvs_by_date(files)
create_time_lagged_features()

Combined data saved to input.d.csv
Engineered features saved to input.engineered.csv


/var/folders/kg/6qgzh1c962vg2xt2n9swng0h0000gn/T/ipykernel_75291/982275056.py:23: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
